In [1]:
# Packages used in the script
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import io
import gzip
import spacy
import re
from contractions import contractions_dict

In [ ]:
df_electro = pd.read_csv (r'Path where the CSV file is stored\df_electronics.csv')

In [12]:
df_electro['tokenize'] = df_electro['sentences'].apply(lambda x: [(token.text) for token in nlp(str(x)) if token.is_alpha if not token.like_url]) 

In [13]:
# Extract noun chunks from the answers
chunk = []
pos = []
 
def noun_chunk(doc):
    for row in doc:
        chunk.append([re.split(' ', str(x)) for x in nlp(str(row)).noun_chunks]) 
    for word in chunk:
        pos.append([(x.text, x.pos_) for x in nlp(str(word)) if x.is_alpha])
                    #if not x.pos_ == 'DET'
                    #if not x.pos_ == 'PRON'
                   # if not x.pos_ == 'ADP'
                    #if not x.pos_ == 'CCONJ'])
                    #if not x.pos_ == 'NUM'])
    return pos
                 
df_electro['noun_chunk'] = noun_chunk(df_electro['sentences'])              

In [15]:
#Remove sentences without a noun_chunk
#Rows before removing empty noun_chunks 416821
df_electro = df_electro[df_electro['noun_chunk'].map(lambda d: len(d)) > 0].reset_index(drop = True)

In [16]:
df_electro['adjective'] = df_electro['noun_chunk'].apply(lambda x: [word for i in x for word in i if 'ADJ' in word])

In [17]:
df_electro = df_electro[df_electro['adjective'].map(lambda d: len(d)) > 0].reset_index(drop = True)


In [18]:
del df_electro['adjective']

In [19]:
# List the positive and negative words
positive_words = []
negative_words = []

with open('positive-words.txt','r') as p:
    for line in p:
        for word in line.split():
            positive_words.append(word)

with open('negative-words.txt', 'r') as n:
    for line in n:
        for word in line.split():
            negative_words.append(word)
            


In [22]:
#df_electro[df_electro['sentences'].str.contains('|'.join(positive_words))]
def opinion_word(x):
    y = []
    for word, tag in x:
        if tag == 'ADJ':
            if word in positive_words:
                y.append((word, 'P'))
            if word in negative_words:
                y.append((word, 'N'))
    return y    
                   
df_electro['opinion_word'] = df_electro['noun_chunk'].apply(opinion_word)
  

In [24]:
df_electro = df_electro[df_electro['opinion_word'].map(lambda d: len(d)) > 0].reset_index(drop = True)

In [25]:
df_electro

,sentences,product_type,tokenize,noun_chunk,opinion_word
0,the nook clips into a secure and safe holder inside that is small and convenient.,0594033926,"[the, nook, clips, into, a, secure, and, safe, holder, inside, that, is, small, and, convenient]","[(the, DET), (nook, VERB), (clips, NOUN), (a, DET), (secure, ADJ), (and, CCONJ), (safe, ADJ), (holder, NOUN)]","[(secure, P), (safe, P)]"
1,this is the best cover i have ever had for my nook- trim and functional (no magnets or elastic needed).,0594033926,"[this, is, the, best, cover, i, have, ever, had, for, my, trim, and, functional, no, magnets, or, elastic, needed]","[(the, DET), (best, ADJ), (cover, NOUN), (i, PRON), (my, PRON), (trim, VERB), (no, INTJ), (magnets, NOUN)]","[(best, P)]"
2,then you download a free and very popular e-book library management program called calibri and a plugin for it that lets it decrypt kindle files.,140053271X,"[then, you, download, a, free, and, very, popular, e, book, library, management, program, called, calibri, and, a, plugin, for, it, that, lets, it, decrypt, kindle, files]","[(you, PRON), (a, DET), (free, ADJ), (and, CCONJ), (very, ADV), (popular, ADJ), (e, NOUN), (book, NOUN), (library, NOUN), (management, NOUN), (program, NOUN), (calibri, NOUN), (a, DET), (plugin, NOUN), (it, PRON), (it, PRON), (kindle, NOUN), (files, NOUN)]","[(free, P), (popular, P)]"
3,some higher end ereaders have the capability to access the internet like a smart phone.,140053271X,"[some, higher, end, ereaders, have, the, capability, to, access, the, internet, like, a, smart, phone]","[(some, DET), (higher, ADJ), (end, NOUN), (ereaders, NOUN), (the, DET), (capability, NOUN), (the, DET), (internet, NOUN), (a, DET), (smart, ADJ), (phone, NOUN)]","[(smart, P)]"
4,"there is a one year limited warranty that comes with the device, right out of the box. beyond that, you can purchase extended warranties and more comprehensive protection plans.",1400698987,"[there, is, a, one, year, limited, warranty, that, comes, with, the, device, right, out, of, the, box, beyond, that, you, can, purchase, extended, warranties, and, more, comprehensive, protection, plans]","[(a, DET), (one, NUM), (year, NOUN), (limited, ADJ), (warranty, NOUN), (the, DET), (device, NOUN), (the, DET), (box, NOUN), (you, PRON), (extended, ADJ), (warranties, NOUN), (more, ADJ), (comprehensive, ADJ), (protection, NOUN), (plans, NOUN)]","[(limited, N), (comprehensive, P)]"
...,...,...,...,...,...
44994,there is a firmware update due out next week that will allow you to rename each sabrent to more usable names - kitchen airplay.,B00L26YDA4,"[there, is, a, firmware, update, due, out, next, week, that, will, allow, you, to, rename, each, sabrent, to, more, usable, names, kitchen, airplay]","[(a, DET), (firmware, NOUN), (update, NOUN), (you, PRON), (each, DET), (sabrent, NOUN), (more, ADJ), (usable, ADJ), (names, NOUN), (kitchen, NOUN), (airplay, NOUN)]","[(usable, P)]"
44995,it has a full version of office and a free antivirus that just had to be updated,B00L3EE1SO,"[it, has, a, full, version, of, office, and, a, free, antivirus, that, just, had, to, be, updated]","[(it, PRON), (a, DET), (full, ADJ), (version, NOUN), (office, NOUN), (a, DET), (free, ADJ), (antivirus, NOUN)]","[(free, P)]"
44996,hummmm now there is good question.,B00L8A94AI,"[hummmm, now, there, is, good, question]","[(good, ADJ), (question, NOUN)]","[(good, P)]"
44997,google them they have amazing customer service.,B00L8A94AI,"[google, them, they, have, amazing, customer, service]","[(them, PRON), (they, PRON), (amazing, ADJ), (customer, NOUN), (service, NOUN)]","[(amazing, P)]"


In [26]:
# Select random 5000 rows 
df_elec = df_electro.sample(n=5000, random_state = 2).reset_index(drop = True)


In [28]:
# Aspect extraction positive aspects

words = []
tags = []


for sentence in df_elec['noun_chunk']:
    for word, tag in sentence:
        aspects_positive = []
        sentiment = []
        words.append(word)
        tags.append(tag)
        
        pattern1_pos = ['ADJ','NOUN']
              
        for p in pattern1_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
        
        for i in range(len(tags)):
            if tags[i:i+len(pattern1_pos)] == pattern1_pos:
                aspects_positive.append(words[i:i+len(pattern1_pos)])
                
               
                 
        pattern2_pos = ['ADJ', 'NOUN', 'NOUN']            
        
        for p in pattern2_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
                    
        for i in range(len(tags)):
            if tags[i:i+len(pattern2_pos)] == pattern2_pos:
                aspects_positive.append(words[i:i+len(pattern2_pos)])
                
               
                    
                    
        pattern3_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern3_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern3_pos)] == pattern3_pos:
                aspects_positive.append(words[i:i+len(pattern3_pos)])
                
                
       
        pattern4_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern4_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern4_pos)] == pattern4_pos:
                aspects_positive.append(words[i:i+len(pattern4_pos)]) 
                
                
        
        pattern5_pos = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern5_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern5_pos)] == pattern5_pos:
                aspects_positive.append(words[i:i+len(pattern5_pos)]) 
                
                
    
        pattern6_pos = ['ADJ', 'VERB']
             
        for p in pattern6_pos:
            if tag == 'ADJ':
                if word not in positive_words:
                    words.append(word)
                    tags.append(word)    
                
        for i in range(len(tags)):
            if tags[i:i+len(pattern6_pos)] == pattern6_pos:
                aspects_positive.append(words[i:i+len(pattern6_pos)])
                
                        
aspects_positive = [' '.join(a) for a in aspects_positive]
sentiment = [' '.join(s) for s in sentiment]



In [29]:
# Remove duplicates 
aspects_positive = list(set(aspects_positive))

aspects_positive = [i for i in aspects_positive if len(i) >= 2]

In [595]:
print(len(aspects_positive))

3871


In [31]:
# Aspect extraction negative aspects

words = []
tags = []


for sentence in df_elec['noun_chunk']:
    for word, tag in sentence:
        aspects_negative = []
        sentiment = []
        words.append(word)
        tags.append(tag)
        
        pattern1_neg = ['ADJ','NOUN']
              
        for p in pattern1_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
        
        for i in range(len(tags)):
            if tags[i:i+len(pattern1_neg)] == pattern1_neg:
                aspects_negative.append(words[i:i+len(pattern1_neg)])
                
                
        pattern2_neg = ['ADJ', 'NOUN', 'NOUN']            
        
        for p in pattern2_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
                    
        for i in range(len(tags)):
            if tags[i:i+len(pattern2_neg)] == pattern2_neg:
                aspects_negative.append(words[i:i+len(pattern2_neg)])
                
                    
                    
        pattern3_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern3_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern3_neg)] == pattern3_neg:
                aspects_negative.append(words[i:i+len(pattern3_neg)])
                
                
       
        pattern4_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern4_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern4_neg)] == pattern4_neg:
                aspects_negative.append(words[i:i+len(pattern4_neg)]) 
                
               
        
        pattern5_neg = ['ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']
        
        for p in pattern5_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)
            
        for i in range(len(tags)):
            if tags[i:i+len(pattern5_neg)] == pattern5_neg:
                aspects_negative.append(words[i:i+len(pattern5_neg)]) 
                
                
    
        pattern6_neg = ['ADJ', 'VERB']
             
        for p in pattern6_neg:
            if tag == 'ADJ':
                if word not in negative_words:
                    words.append(word)
                    tags.append(word)    
                
        for i in range(len(tags)):
            if tags[i:i+len(pattern6_neg)] == pattern6_neg:
                aspects_negative.append(words[i:i+len(pattern6_neg)])
                
                        
aspects_negative = [' '.join(a) for a in aspects_negative]
sentiment = [' '.join(s) for s in sentiment]

In [32]:
# Remove duplicates 
aspects_negative = list(set(aspects_negative))

aspects_negative = [i for i in aspects_negative if len(i) >= 2]

In [596]:
print(len(aspects_negative))

1135


In [34]:
# Concatenate the two lists togehter
aspects = [y for x in [aspects_positive, aspects_negative] for y in x]

In [597]:
print(len(aspects))

5006


In [36]:
# Match the aspect words with the sentence 
#df_elec['aspect_word'] = df_elec['sentences'].str.findall(r"\b"+"|".join([i.split() for i in str(aspects)]) + r"\b")#.str.join('')

df_elec['aspect_word'] = df_elec['sentences'].str.findall(r"\b"+"|".join(aspects) + r"\b").str.join(' ')





In [37]:
df_elec['aspect_word'] = df_elec['aspect_word'].str.split()

In [38]:
df_elec

,sentences,product_type,tokenize,noun_chunk,opinion_word,aspect_word
0,express scribe is a free software.,B008EA1K66,"[express, scribe, is, a, free, software]","[(express, NOUN), (scribe, VERB), (a, DET), (free, ADJ), (software, NOUN)]","[(free, P)]","[free, software]"
1,this camera was voted the best dslr camera on wire magazine.,B000NVXF26,"[this, camera, was, voted, the, best, dslr, camera, on, wire, magazine]","[(this, DET), (camera, NOUN), (the, DET), (best, ADJ), (dslr, NOUN), (camera, NOUN), (wire, NOUN), (magazine, NOUN)]","[(best, P)]","[best, dslr]"
2,"there will be problems down the line and if they do not have clue one how to begin to fix these themselves, they will end up frustrated, paying out more money then they anticipated, and will have very expensive door stops on their hands.",B009AOFMH8,"[there, will, be, problems, down, the, line, and, if, they, do, not, have, clue, one, how, to, begin, to, fix, these, themselves, they, will, end, up, frustrated, paying, out, more, money, then, they, anticipated, and, will, have, very, expensive, door, stops, on, their, hands]","[(problems, NOUN), (the, DET), (line, NOUN), (they, PRON), (clue, NOUN), (these, DET), (themselves, PRON), (they, PRON), (more, ADJ), (money, NOUN), (they, PRON), (very, ADJ), (expensive, ADJ), (door, NOUN), (their, PRON), (hands, NOUN)]","[(expensive, N)]","[expensive, door]"
3,as much as you can try to use a 70g200 or a 18 to 200 mm if you use sony or above they will not be as cheap option as this one but to be honest this kind of lens is kind of going back to the 70`s and the definition you will get will not be as good and you will have to manually fine tune for each shot.,B00DOAQ7VY,"[as, much, as, you, can, try, to, use, a, or, a, to, mm, if, you, use, sony, or, above, they, will, not, be, as, cheap, option, as, this, one, but, to, be, honest, this, kind, of, lens, is, kind, of, going, back, to, the, and, the, definition, you, will, get, will, not, be, as, good, and, you, will, have, to, manually, fine, tune, for, each, shot]","[(you, PRON), (mm, NOUN), (you, PRON), (sony, NOUN), (they, PRON), (cheap, ADJ), (option, NOUN), (this, DET), (one, NUM), (this, DET), (kind, NOUN), (lens, NOUN), (the, DET), (the, DET), (definition, NOUN), (you, PRON), (you, PRON), (to, PART), (manually, ADV), (fine, ADJ), (tune, NOUN), (each, DET), (shot, NOUN)]","[(cheap, N), (fine, P)]","[cheap, option, fine, tune]"
4,80 gb sata 3.5 hard drive,B002YAFRWU,"[gb, sata, hard, drive]","[(gb, VERB), (sata, NOUN), (hard, ADJ), (drive, VERB)]","[(hard, N)]","[hard, drive]"
...,...,...,...,...,...,...
4995,desk top tripods that could be used to make this a very valuable feature (i never have used mine)but realy forgot it was there ... so thank you ...,B004YW7WCY,"[desk, top, tripods, that, could, be, used, to, make, this, a, very, valuable, feature, i, never, have, used, realy, forgot, it, was, there, so, thank, you]","[(top, NOUN), (tripods, NOUN), (this, DET), (a, DET), (very, ADV), (valuable, ADJ), (feature, NOUN), (i, PRON), (realy, NOUN), (it, PRON), (you, PRON)]","[(valuable, P)]","[valuable, feature]"
4996,"i thinks it is a nice, small, sturdy music player, with surprising sound.",B00HQO34HY,"[i, thinks, it, is, a, nice, small, sturdy, music, player, with, surprising, sound]","[(i, PRON), (it, PRON), (a, DET), (nice, ADJ), (small, ADJ), (sturdy, ADJ), (music, NOUN), (player, NOUN), (surprising, ADJ), (sound, NOUN)]","[(nice, P), (sturdy, P)]","[sturdy, music, player]"
4997,good luck with yours.,B00FO4GVAK,"[good, luck, with, yours]","[(good, ADJ), (luck, NOUN), (yours, PRON)]","[(good, P)]","[good, luck]"
4998,it's a very powerful amp with true wattage so it should do exactly what you are looking for.,B00716308A,"[it, a, very, powerful, amp, with, true, wattage, so, it, should, do, exactly, what, you, are, looking, for]","[(it, PRON), (a, DET), (very, ADV), (powerful, ADJ), (amp, NOUN), (true, ADJ), (wattage, NOUN), (it, PRON), (exactly, ADV), (what, PRON), (you, PRO

In [39]:
# IOB tagging
aspect = [x.split()[0:3] for x in aspects]
pos = [x.split()[0:3] for x in aspects_positive]
neg = [x.split()[0:3] for x in aspects_negative]
p = [x[0] for x in pos]
n = [x[0] for x in neg]   
beginning_chunk_positive = [x[1] for x in pos]
beginning_chunk_negative = [x[1] for x in neg]
remaining_chunk_positive = [x[-1] for x in pos]
remaining_chunk_negative = [x[-1] for x in neg]
#remaining_chunk = [x for x in remaining_words if x]


def IOB_tagging(sentence):  
    tags =[]
    opinion_p = []
    opinion_n = []
    chunk_p = []
    chunk_n = []
    
    for word in sentence:
        try:
            if word in p:
                opinion_p = word
                tags.append('O')
            elif word in beginning_chunk_positive and (word[-1] in opinion_p or word[1] in opinion_p):
                chunk_p = word
                tags.append('B_pos')
            elif word in remaining_chunk_positive and word[-2] in chunk_p:
                tags.append('I_pos')
            elif word in n:
                opinion_n = word
                tags.append('O')
            elif word in beginning_chunk_negative and (word[-1] in opinion_n or word[1] in opinion_n):
                chunk_n = word
                tags.append('B_neg')
            elif word in remaining_chunk_negative and word[-2] in chunk_n:
                tags.append('I_neg')
            else:
                tags.append('O')
        except IndexError:
            pass
    
    return tags

df_elec['IOB'] = df_elec['tokenize'].apply(IOB_tagging)

In [40]:
df_elec[0:30]

,sentences,product_type,tokenize,noun_chunk,opinion_word,aspect_word,IOB
0,express scribe is a free software.,B008EA1K66,"[express, scribe, is, a, free, software]","[(express, NOUN), (scribe, VERB), (a, DET), (free, ADJ), (software, NOUN)]","[(free, P)]","[free, software]","[O, O, O, O, O, B_pos]"
1,this camera was voted the best dslr camera on wire magazine.,B000NVXF26,"[this, camera, was, voted, the, best, dslr, camera, on, wire, magazine]","[(this, DET), (camera, NOUN), (the, DET), (best, ADJ), (dslr, NOUN), (camera, NOUN), (wire, NOUN), (magazine, NOUN)]","[(best, P)]","[best, dslr]","[O, O, O, O, O, O, B_pos, I_pos, O, B_pos, O]"
2,"there will be problems down the line and if they do not have clue one how to begin to fix these themselves, they will end up frustrated, paying out more money then they anticipated, and will have very expensive door stops on their hands.",B009AOFMH8,"[there, will, be, problems, down, the, line, and, if, they, do, not, have, clue, one, how, to, begin, to, fix, these, themselves, they, will, end, up, frustrated, paying, out, more, money, then, they, anticipated, and, will, have, very, expensive, door, stops, on, their, hands]","[(problems, NOUN), (the, DET), (line, NOUN), (they, PRON), (clue, NOUN), (these, DET), (themselves, PRON), (they, PRON), (more, ADJ), (money, NOUN), (they, PRON), (very, ADJ), (expensive, ADJ), (door, NOUN), (their, PRON), (hands, NOUN)]","[(expensive, N)]","[expensive, door]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,as much as you can try to use a 70g200 or a 18 to 200 mm if you use sony or above they will not be as cheap option as this one but to be honest this kind of lens is kind of going back to the 70`s and the definition you will get will not be as good and you will have to manually fine tune for each shot.,B00DOAQ7VY,"[as, much, as, you, can, try, to, use, a, or, a, to, mm, if, you, use, sony, or, above, they, will, not, be, as, cheap, option, as, this, one, but, to, be, honest, this, kind, of, lens, is, kind, of, going, back, to, the, and, the, definition, you, will, get, will, not, be, as, good, and, you, will, have, to, manually, fine, tune, for, each, shot]","[(you, PRON), (mm, NOUN), (you, PRON), (sony, NOUN), (they, PRON), (cheap, ADJ), (option, NOUN), (this, DET), (one, NUM), (this, DET), (kind, NOUN), (lens, NOUN), (the, DET), (the, DET), (definition, NOUN), (you, PRON), (you, PRON), (to, PART), (manually, ADV), (fine, ADJ), (tune, NOUN), (each, DET), (shot, NOUN)]","[(cheap, N), (fine, P)]","[cheap, option, fine, tune]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_neg, O, O, O, O, O, O, O, O, O, O, B_pos, O, I_pos, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_pos, O, O, B_neg]"
4,80 gb sata 3.5 hard drive,B002YAFRWU,"[gb, sata, hard, drive]","[(gb, VERB), (sata, NOUN), (hard, ADJ), (drive, VERB)]","[(hard, N)]","[hard, drive]","[O, O, O, B_neg]"
5,good question..it happened to me too.,B004D7JRA0,"[good, question, it, happened, to, me, too]","[(good, ADJ), (question, NOUN), (it, PRON), (me, PRON)]","[(good, P)]","[good, question]","[O, O, O, O, O, O, O]"
6,"i believe they are interchangeable......wonderful, warm sound.....fabulous speakers.....though i traded up for focals.",B00422L39Y,"[i, believe, they, are, interchangeable, wonderful, warm, sound, fabulous, speakers, though, i, traded, up, for, focals]","[(i, PRON), (they, PRON), (fabulous, ADJ), (speakers, NOUN), (i, PRON), (focals, NOUN)]","[(fabulous, P)]","[fabulous, speakers]","[O, O, O, O, O, O, O, O, O, B_pos, O, O, O, O, O, O]"
7,i felt that in most regards it was superior lens; but considering the price difference it was the clear choice for me.,B0014Z3XMC,"[i, felt, that, in, most, regards, it, was, superior, lens, but, considering, the, price, difference, it, was, the, clear, choice, for, me]","[(i, PRON), (regards, NOUN), (it, PRON), (sup

In [127]:
# Export DataFrame to CSV file
df_elec.to_csv(r'/Path where the CSV file is stored/df_electronics.csv', index = False)